In [1]:
import pandas as pd
from scipy.stats import qmc
import random
import time
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init
import itertools

In [2]:
qs = {
    "methanol" : 1.4320,
    "ethanol" : 2.5880,
    "2-propanol" : 3.1240,
    "acetone" : 2.3360,
    "mibk" : 3.9520,
    "ethyl-acetate" : 3.1160,
    "chloroform" : 2.41,
    "toluene" : 2.9680,
    "water" : 1.4,
    "n-pentane" : 3.316,
    "n-heptane" : 4.396,
    "1-propanol" : 3.128,
    "1-pentanol" : 4.208,
    "1-butanol" : 3.668,
}

rs = {
    "methanol" : 1.4311,
    "ethanol" : 2.5755,
    "2-propanol" : 3.2491,
    "acetone" : 2.5735,
    "mibk" : 4.5959,
    "ethyl-acetate" : 3.4786,
    "chloroform" : 2.87,
    "toluene" : 3.9228,
    "water" : 0.92,
    "n-pentane" : 3.8254,
    "n-heptane" : 5.1742,
    "1-propanol" : 3.2499,
    "1-pentanol" : 4.5987,
    "1-butanol" : 3.9243,
}

In [3]:
inv_qs = dict(map(reversed, qs.items()))

In [4]:
methanol = component(name='methanol', Tc=512.5, Pc=80.84, Zc=0.222, Vc=117, GC={'CH3OH':1})
ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168, GC={'CH3':1, 'CH2':1, 'OH':1})
_2propanol = component(name='2-propanol', Tc=508.3, Pc=47.65, Zc=0.25, Vc=222, GC={'CH3':2, 'CH2':1, 'OH':1})
acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213, GC={'CH3CO':1, 'CH3':1})
mibk = component(name='mibk', Tc=574.6, Pc=32.7, Zc=0.256, Vc=317, GC={'CH3CO':1, 'CH3':2, 'CH2':1, 'CH':1})
ethac = component(name='ethyl-acetate', Tc=523.3, Pc=38.8, Zc=0.255, Vc=286, GC={'CH3COO':1, 'CH3':1, 'CH2':1})
chloroform = component(name='chloroform', Tc=536.4, Pc=54.72, Zc=0.293, Vc=239, GC={'CHCL3':1})
toluene = component(name='toluene', Tc=591.75, Pc=41.08, Zc=0.264, Vc=316, GC={'ACH':5, 'ACCH3':1})
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,GC={'H2O':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1})

In [5]:
component_list = [methanol,ethanol,_2propanol,acetone,mibk,ethac,chloroform,toluene,water]

In [6]:
binary_mix = itertools.combinations(component_list, 2) #outputs every combination of mixtures give a list

In [7]:
binary_mix= list(binary_mix)
binary_mix_no = len(binary_mix) #number of different mixtures
print('binary combinations', binary_mix_no)

n=0
binary_mix_list = [] #list of binary mixtures

while n <= (binary_mix_no-1):
    mix = mixture(binary_mix[n][0],binary_mix[n][1])
    #print('mixture index:',n, 'Component names:', mix.names)
    binary_mix_list.append(mix)
    n=n+1

binary combinations 36


In [8]:
num_samples_base2 = 9

In [9]:
st = time.time() #start timer
sampler_sob = qmc.Sobol(d=3, scramble=True)
sample_sob = sampler_sob.random_base2(num_samples_base2)
final_sample_list = []
for sample in sample_sob:
    x1 = sample[0]/(sample[0] + sample[1] + sample[2])
    x2 = sample[1]/(sample[0] + sample[1] + sample[2])
    x3 = sample[2]/(sample[0] + sample[1] + sample[2])
    Tsob = 300
    final_sample_list.append([x1, x2, Tsob])
et = time.time()
sobol_time = et - st
print('Time taken for Sobol sequence generation: ', sobol_time)

Time taken for Sobol sequence generation:  0.02703261375427246


In [10]:
st = time.time()
master_list = []
for i in binary_mix:
    s1 = i[0].name
    s2 = i[1].name
    q1 = qs[s1]
    q2 = qs[s2]
    r1 = rs[s1]
    r2 = rs[s2]
    for j in final_sample_list:
        master_list.append([s1,s2,q1,q2,r1,r2,j[0],j[1],j[2]])
et = time.time()
list_time = et - st
print('Time taken for list generation: ', list_time)

Time taken for list generation:  0.00657963752746582


In [11]:
master_list_column_names = ['Solvent 1', 'Solvent 2', 'q1', 'q2', 'r1', 'r2', 'x1', 'x2', 'T']
df = pd.DataFrame(master_list, columns=master_list_column_names)
df.to_csv('Case1.csv')
print('Total time for generating data: ', sobol_time + list_time)

Total time for generating data:  0.03361225128173828
